# Stage 4

- has **`location_raw`**: True
- vars one-hot encoded: False
- var label-encoded: True
- oversampled: True

In [2]:
from datetime import datetime
import pickle

from sklearn.model_selection import RandomizedSearchCV, train_test_split
import humanize
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier



%matplotlib inline
%load_ext autoreload

In [3]:
X_train = pickle.load(open('./data/stage4-train.pkl', 'rb'))
y_train = X_train.pop('stop_outcome')
X_test = pickle.load(open('./data/stage4-test.pkl', 'rb'))
y_test = X_test.pop('stop_outcome')

print('Row counts:\n\ttrain: {}\n\ttest: {}'.format(X_train.shape[0], X_test.shape[0]))

Row counts:
	train: 628906
	test: 55547


In [3]:
# features = pickle.load(open('./data/stage4-l_encoded-oversampled.pkl', 'rb'))
# labels = features.pop('stop_outcome')

In [4]:
# Take out 5% of data for final final testing; shuffle first
# final_test_features = pickle.load(open('./data/final_test_set.pkl', 'rb'))
# final_test_outcomes = final_test_features.pop('stop_outcome')
# print('Final test set row count: {}'.format(final_test_features.shape[0]))

# X_train = features
# y_train = labels
# X_test = final_test_features
# y_test = final_test_outcomes

# Simple Linear Classifier

In [5]:
from sklearn import linear_model

clf_sgd = linear_model.SGDClassifier()
clf_sgd.fit(X_train, y_train)
clf_sgd.score(X_test, y_test)

/home/pato/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.68118890309107605

# VotingClassifier

In [6]:
clf1 = RandomForestClassifier(n_jobs=8, verbose=3, random_state=0)
clf2 = GaussianNB()
clf3 = DecisionTreeClassifier(random_state=0)
clf4 = GradientBoostingClassifier(verbose=3, random_state=0)

eclf = VotingClassifier(estimators=[('rf', clf1), ('gnb', clf2), ('dt', clf3), ('gb', clf4)],
                        voting='soft')
eclf = eclf.fit(X_train, y_train)
print('eclf score: {}'.format(eclf.score(X_test, y_test)))

building tree 1 of 10building tree 2 of 10building tree 3 of 10building tree 4 of 10building tree 5 of 10building tree 6 of 10




building tree 7 of 10
building tree 8 of 10

building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=8)]: Done   3 out of  10 | elapsed:    1.8s remaining:    4.3s
[Parallel(n_jobs=8)]: Done   7 out of  10 | elapsed:    2.0s remaining:    0.8s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    2.7s finished


      Iter       Train Loss   Remaining Time 
         1      966737.1600            4.37m
         2      938521.2147            4.36m
         3      915793.7583            4.31m
         4      897012.8253            4.26m
         5      881475.0663            4.22m
         6      868541.5927            4.18m
         7      857503.3246            4.14m
         8      848309.0173            4.11m
         9      840114.7978            4.06m
        10      833249.3731            4.02m
        11      827302.9631            3.98m
        12      821642.0822            3.94m
        13      817122.6466            3.88m
        14      812900.7590            3.84m
        15      809349.9902            3.79m
        16      806250.7984            3.74m
        17      803465.3690            3.69m
        18      800763.7098            3.65m
        19      797216.7401            3.62m
        20      794977.1946            3.57m
        21      793149.3354            3.53m
        2

[Parallel(n_jobs=8)]: Done   3 out of  10 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   7 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished


eclf score: 0.6084577024861829


In [7]:
eclf.estimators_

[RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=8,
             oob_score=False, random_state=0, verbose=3, warm_start=False),
 GaussianNB(priors=None),
 DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=0,
             splitter='best'),
 GradientBoostingClassifier(criterion='friedman_mse', init=None,
               learning_rate=0.1, loss='deviance', max_depth=3,
               max_features=None, max_leaf_nodes=None,
               min_impu

# GaussianNB

In [8]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
print('{}'.format(gnb.score(X_test, y_test)))

0.6537166723675446


# DecisionTreeClassifier

In [9]:
dtc = DecisionTreeClassifier(random_state=0)
dtc.fit(X_train, y_train)
print('{}'.format(dtc.score(X_test, y_test)))

0.5296595675734063


# RandomForestClassifier

In [10]:
rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_train, y_train)
print('{}'.format(rfc.score(X_test, y_test)))

0.5750985651790376


# GradientBoostingClassifier

In [11]:
gbc = GradientBoostingClassifier(random_state=0)
gbc.fit(X_train, y_train)
print('{}'.format(gbc.score(X_test, y_test)))

0.5244927718868706


## GradientBoostingClassifier (Tuned)


In [4]:
gbc_tuned = GradientBoostingClassifier(
    learning_rate=0.0983,
    max_depth=6,
    max_features=len(list(X_train.columns.values)),
    subsample=0.9,
    verbose=3,
    random_state=0,
)
gbc_tuned.fit(X_train, y_train)
print('{}'.format(gbc_tuned.score(X_test, y_test)))

      Iter       Train Loss      OOB Improve   Remaining Time 
         1      793889.6061       11425.9968          796.86m
         2      713850.9406        8394.0481          959.58m


KeyboardInterrupt: 